In [1]:
# CLI
# GUI

# Terminal, CMD

# SSH, SCP 

# tar, gzip, tar.gz, zip...
# compression format

# tar -> 여러 파일을 1개파일로 묶음. 그냥 묶기
# gzip -> 1개 파일을 압축을 함
# tar.gz -> tar + gzip, tar로 1개로 묶고, 그다음 gzip으로 압축
# zip -> 여러 파일을 한번에 압축을 함

# Configuration : XML, JSON, YAML, Dataclass, Pydantic (id, pw나 기타 등등 자주 바뀌는 변수들을 담아놓은 별도의 공간) 

# 위 키워드로 검색해서 구글에서 많이 읽어보세요~~
# 특히 SSH, SCP 명령어 잘 기억해놓으세요

In [ ]:
# PLP package

# Server 
# Unzip
# Build
# Code to Run -> configuration
# run X
# Results -> local-> or shiny

# Purpose : 명령어 한줄로 우리가 원하는 PLP zip파일을 우리가 원하는 서버에서 돌리고, 결과만 받으면 좋겠다.


In [7]:
# 1. SSH connect to server

#pip install paramiko
#pip install scp

import paramiko
from scp import SCPClient
import getpass

def createSSHClient(server, port, user, password):
    client = paramiko.SSHClient()
    client.load_system_host_keys()
    client.set_missing_host_key_policy(paramiko.AutoAddPolicy())
    client.connect(server, port, user, password)
    return client

pw = getpass.getpass()
ssh = createSSHClient('10.5.12.43', '22', 'cbj', pw)

In [8]:
# SCP
scp = SCPClient(ssh.get_transport())
base_path = '/home/cbj/ssh_test'
scp.put('fltest3.zip',base_path)

In [9]:
# unzip

stdin, stdout, stderr=ssh.exec_command(f'unzip -o {base_path}/fltest3.zip')


In [10]:
lines = stdout.readlines()
lines

['Archive:  /home/cbj/ssh_test/fltest3.zip\n',
 '  inflating: .Rbuildignore           \n',
 '  inflating: .Rprofile               \n',
 '  inflating: .Rproj.user/F934BCE8/sources/prop/53F8A3DD  \n',
 '  inflating: .Rproj.user/F934BCE8/sources/prop/INDEX  \n',
 '  inflating: .Rproj.user/F934BCE8/sources/s-3C059F61/F2D43E04  \n',
 '  inflating: .Rproj.user/F934BCE8/sources/s-3C059F61/F2D43E04-contents  \n',
 '  inflating: .Rproj.user/F934BCE8/sources/s-3C059F61/lock_file  \n',
 '  inflating: .Rproj.user/shared/notebooks/patch-chunk-names  \n',
 '  inflating: .gitignore              \n',
 '  inflating: DESCRIPTION             \n',
 '  inflating: HydraConfig.json        \n',
 '  inflating: NAMESPACE               \n',
 '  inflating: R/CreateAllCohorts.R    \n',
 '  inflating: R/CreateCohorts.R       \n',
 '  inflating: R/Main.R                \n',
 '  inflating: R/PackageResults.R      \n',
 '  inflating: R/fltest3.R             \n',
 '  inflating: R/SubmitResults.R       \n',
 '  inflatin

In [11]:
# build
build_command = (
	f"R CMD INSTALL --no-multiarch --with-keep.source {base_path}"
)
stdin, stdout, stderr=ssh.exec_command(build_command)
lines = stdout.readlines()
lines

['   ‘UsingSkeletonPackage.Rmd’ using ‘UTF-8’ \n']

In [18]:
# configuration
from omegaconf import OmegaConf
db_conf = OmegaConf.load('client_conf.yaml').db_conf

In [20]:
# Make script
project= 'fltest3'
rscript=f'''

library({project})
library(dplyr)
# USER INPUTS
#=======================
# The folder where the study intermediate and result files will be written:
outputFolder <- "Results"

# Specify where the temporary files (used by the ff package) will be created:
options(fftempdir = "./temp")

# Details for connecting to the server:
dbms <- "{db_conf.dbms}"
user <- '{db_conf.user}'
pw <- '{db_conf.pw}'
server <- '{db_conf.server}'
port <- '{db_conf.port}' 

downloadJdbcDrivers('{db_conf.dbms}','./{project}')
connectionDetails <- DatabaseConnector::createConnectionDetails(dbms = dbms,
                                                            server = server,
                                                            user = user,
                                                            password = pw,
                                                            port = port,
                                                            pathToDriver='./{project}')

# Add the database containing the OMOP CDM data
cdmDatabaseSchema <- '{db_conf.cdmDatabaseSchema}'
# Add a sharebale name for the database containing the OMOP CDM data
cdmDatabaseName <- '{db_conf.cdmDatabaseName}'
# Add a database with read/write access as this is where the cohorts will be generated
cohortDatabaseSchema <- '{db_conf.cohortDatabaseSchema}'

oracleTempSchema <- NULL

# table name where the cohorts will be generated
cohortTable <- '{project}Cohort'
#=======================

execute(connectionDetails = connectionDetails,
    cdmDatabaseSchema = cdmDatabaseSchema,
    cdmDatabaseName = cdmDatabaseName,
    cohortDatabaseSchema = cohortDatabaseSchema,
    oracleTempSchema = oracleTempSchema,
    cohortTable = cohortTable,
    outputFolder = outputFolder,
    createProtocol = F,
    createCohorts = T,
    runAnalyses = T,
    createResultsDoc = F,
    packageResults = F,
    createValidationPackage = F,  
    #analysesToValidate = 1,
    minCellCount= 5,
    createShiny = F,
    createJournalDocument = F,
    analysisIdDocument = 1)
'''

with open('./temp_run.R','w') as f:
    f.write(rscript)



In [21]:
# transport script
scp = SCPClient(ssh.get_transport())
base_path = '/home/cbj/ssh_test'
scp.put('temp_run.R',f'{base_path}/extras/')

In [22]:
# run Rscript
stdin, stdout, stderr=ssh.exec_command(f"Rscript {base_path}/extras/temp_run.R")
lines = stdout.readlines()
print(lines)


['Creating cohorts\n', 'Creating cohort: cbj_fl_outcome\n', '\r  |                                                                            \r  |                                                                      |   0%\r  |                                                                            \r  |===                                                                   |   5%\r  |                                                                            \r  |=======                                                               |  10%\r  |                                                                            \r  |==========                                                            |  14%\r  |                                                                            \r  |=============                                                         |  19%\r  |                                                                            \r  |=================                          

In [25]:
# get results
scp.get(f'{base_path}/extras','.', recursive =True)

In [ ]:
###########################

In [ ]:
def createSSHClient(server, port, user, password):
    client = paramiko.SSHClient()
    client.load_system_host_keys()
    client.set_missing_host_key_policy(paramiko.AutoAddPolicy())
    client.connect(server, port, user, password)
    return client


def plp_run(ssh,base_path, project, db_conf):
    
    rscript=f'''

    library({project})
    library(dplyr)
    # USER INPUTS
    #=======================
    # The folder where the study intermediate and result files will be written:
    outputFolder <- "Results"

    # Specify where the temporary files (used by the ff package) will be created:
    options(fftempdir = "./temp")

    # Details for connecting to the server:
    dbms <- "{db_conf.dbms}"
    user <- '{db_conf.user}'
    pw <- '{db_conf.pw}'
    server <- '{db_conf.server}'
    port <- '{db_conf.port}'

    downloadJdbcDrivers('{db_conf.dbms}','.')
    connectionDetails <- DatabaseConnector::createConnectionDetails(dbms = dbms,
                                                                server = server,
                                                                user = user,
                                                                password = pw,
                                                                port = port,
                                                                pathToDriver='.')

    # Add the database containing the OMOP CDM data
    cdmDatabaseSchema <- '{db_conf.cdmDatabaseSchema}'
    # Add a sharebale name for the database containing the OMOP CDM data
    cdmDatabaseName <- '{db_conf.cdmDatabaseName}'
    # Add a database with read/write access as this is where the cohorts will be generated
    cohortDatabaseSchema <- '{db_conf.cohortDatabaseSchema}'

    oracleTempSchema <- NULL

    # table name where the cohorts will be generated
    cohortTable <- '{project}Cohort'
    #=======================

    execute(connectionDetails = connectionDetails,
        cdmDatabaseSchema = cdmDatabaseSchema,
        cdmDatabaseName = cdmDatabaseName,
        cohortDatabaseSchema = cohortDatabaseSchema,
        oracleTempSchema = oracleTempSchema,
        cohortTable = cohortTable,
        outputFolder = outputFolder,
        createProtocol = F,
        createCohorts = T,
        runAnalyses = T,
        createResultsDoc = F,
        packageResults = F,
        createValidationPackage = F,  
        #analysesToValidate = 1,
        minCellCount= 5,
        createShiny = F,
        createJournalDocument = F,
        analysisIdDocument = 1)
    '''

    with open('./temp_run.R','w') as f:
        f.write(rscript)
    
    scp = SCPClient(ssh.get_transport())
    scp.put(f'{project}.zip',base_path)
    
    unzip_command = f'unzip -o {base_path}/{project}.zip'
    stdin, stdout, stderr=ssh.exec_command(unzip_command)
    lines = stdout.readlines()
    print(lines)
    
    build_command = (
	f"R CMD INSTALL --no-multiarch --with-keep.source {base_path}"
    )
    stdin, stdout, stderr=ssh.exec_command(build_command)
    lines = stdout.readlines()
    print(lines)
    
    scp = SCPClient(ssh.get_transport())
    scp.put('temp_run.R',f'{base_path}/extras/')

    rscript_command = f"Rscript {base_path}/extras/temp_run.R"
    stdin, stdout, stderr=ssh.exec_command(rscript_command)
    lines = stdout.readlines()
    print(lines)

    scp.get(f'{base_path}/extras/Results','.', recursive =True)

    # json file, csv file, rds file
    # 결과값이 바로 출력 HR or N수 출력 

In [27]:
import paramiko
from scp import SCPClient
from omegaconf import OmegaConf
from getpass import getpass

pw = getpass()

ssh = createSSHClient('10.5.12.43', '22', 'cbj', pw)
# configuration
db_conf = OmegaConf.load('client_conf.yaml').db_conf
base_path = '/home/cbj/ssh_test'

plp_run(ssh=ssh,base_path=base_path, project='fltest3', db_conf=db_conf)

In [28]:
# seol_hnnc PLP zipfile -> 
plp_run(ssh=ssh,base_path='/home/cbj/Rtest', project='seol_hnnc', db_conf=db_conf)

In [30]:
# seol_hnnc PLP zipfile -> 
plp_run(ssh=ssh,base_path='/home/cbj/Rtest', project='seol_hnnc', db_conf=db_conf)

In [ ]:
# Webpage에서 PLP 다운 받는 방법
# + 
# plp_run

# 코드 한줄로....원하는 PLP 패키지를 다운받고, plp_run 파이썬 함수를 제작 가능

In [ ]:
# 숙제 : 
# 	다운로드 받은 패키지를 서버로 옮겨서 실행해서 결과까지 확인하는 코드 작성
#	이전 시간에 작성한 Feedernet or ATLAS 코드와 이어지도록 코드 작성해서 순수 파이썬으로 패키지 다운받고 서버로 보내서 실행해서 결과값 받을 수 있도록 코드 작성
#-----

# 다음 수업 2~3주 후...
# 여러 병원에 특정 컨셉에 RC를 확인하기 위해서, feedernet 병원별로 들어가서, concept으로 들어가서, 그 search 패널에 들어가서, 하나하나 쳐봐서 RC를.
# 특정 코호트가 어느 병원에 존재하는지 확인하기 위해서...그 병원에 또 들어